## Basic gradio app

In [16]:
import pandas as pd
import numpy as np
import os
import gradio as gr

from openai.embeddings_utils import distances_from_embeddings

from dotenv import load_dotenv
import os
import openai

load_dotenv()

# this openai api key should be saved in a local file named ".env". It should also be put in the .gitignore to avoid sharing secrets
openai.api_key =  os.getenv("OPENAI_API_KEY")

df=pd.read_csv('processed/embeddings.csv', index_col=0)
# df.to_parquet('processed/embeddings.parquet')


df['embeddings'] = df['embeddings'].apply(eval).apply(np.array)

df.head()

,text,n_tokens,embeddings
0,com insights articles tax salt the latest impa...,17,"[-0.02142474427819252, -0.0044760936871171, 0...."
1,"Cookle, Jessica L. Macklin On June 21...",464,"[0.0025488201063126326, -0.017141442745923996,..."
2,To the extent that businesses sell to end-user...,485,"[0.0018098383443430066, -0.032503217458724976,..."
3,Wipfli’s seasoned tax professionals deliver a ...,493,"[0.006133987568318844, -0.02812495082616806, 0..."
4,com insights articles aa sba small business re...,14,"[-0.004011235199868679, -0.011093624867498875,..."


In [17]:
def create_context(
    question, df, max_len=1800, size="ada"
):
    """
    Create a context for a question by finding the most similar context from the dataframe
    """

    # Get the embeddings for the question
    q_embeddings = openai.Embedding.create(input=question, engine='text-embedding-ada-002')['data'][0]['embedding']

    # Get the distances from the embeddings
    df['distances'] = distances_from_embeddings(q_embeddings, df['embeddings'].values, distance_metric='cosine')


    returns = []
    cur_len = 0

    # Sort by distance and add the text to the context until the context is too long
    for i, row in df.sort_values('distances', ascending=True).iterrows():
        
        # Add the length of the text to the current length
        cur_len += row['n_tokens'] + 4
        
        # If the context is too long, break
        if cur_len > max_len:
            break
        
        # Else add it to the text that is being returned
        returns.append(row["text"])

    # Return the context
    return "\n\n###\n\n".join(returns)

def answer_question(
    df,
    model="text-davinci-003",
    question="Am I allowed to publish model outputs to Twitter, without a human review?",
    max_len=1800,
    size="ada",
    debug=False,
    max_tokens=150,
    stop_sequence=None,
    temperature = 0.
):
    """
    Answer a question based on the most similar context from the dataframe texts
    """
    context = create_context(
        question,
        df,
        max_len=max_len,
        size=size,
    )
    # If debug, print the raw model response
    if debug:
        print("Context:\n" + context)
        print("\n\n")

    try:
        # Create a completions using the question and context
        response = openai.Completion.create(
            prompt=f"Answer the question in a convincing manner based on the context below, and if the question can't be answered based on the context, say \"I don't know\"\n\nContext: {context}\n\n---\n\nQuestion: {question}\nAnswer:",
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=stop_sequence,
            model=model,
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""
    


In [18]:
# https://dev.to/feranmiodugbemi/how-to-create-an-ai-powered-chatbot-with-gradio-and-openais-gpt-35-2c8l

In [23]:
##### import gradio as gr

def text_to_text(input):
    if input:
        question = input
        ans = answer_question(df, question=question)
        return ans
    
def text_to_text(temp, text):
    if text:
        ans = answer_question(df, question=text, temperature=temp)
        return ans
    

inputs = [gr.components.Slider(value = 0., minimum=0., maximum=1., step = 0.05, label="Temperature (0=most factual, 1=most creative)")
          , gr.components.Textbox(lines=4, label="Chat with Wipfli")]
outputs = gr.components.Textbox(label="Output")
examples = [[0.0, "Where are wipfli's largest offices?"],
 [1.0, "Where are wipfli's wisconsin offices?"],
 [0.25, 'Can wipfli help me with BI Services?'],
 [0.9,  'How can wipfli help me with BI Services for my manufacturing company?'],
 # [0.75, 'What is one Wipfli success story?'],
 [0.0, 'How do I get in touch with Wipfli?']
]



In [24]:
print(answer_question(df, question="What services does wipfli provide?",temperature=0.))
print('*'*80)
print(answer_question(df, question="What services does wipfli provide?",temperature=1.))

Wipfli provides a range of services including audit and accounting, CPA tax services, business consulting services, outsourcing, and private client services.
********************************************************************************
Wipfli provides audit and accounting services, CPA tax services, business consulting services, outsourcing services, and private client services.


In [25]:
gr.Interface(fn=text_to_text, inputs=inputs, outputs=outputs, title="wipfli website q&a",
             description="Ask any questions about Wipfli (public website)", examples=examples,
             theme="compact").launch(share=False)


/Users/yarribryn/mambaforge/envs/openai/lib/python3.11/site-packages/gradio/blocks.py:491: UserWarning: Theme should be a class loaded from gradio.themes
  warnings.warn("Theme should be a class loaded from gradio.themes")


Running on local URL:  http://127.0.0.1:7926

To create a public link, set `share=True` in `launch()`.
